In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_excel('../Тестовые/corporate_links_raw.xlsx')

In [6]:
df.head()

,"FIO_owner,Company_name,INN_company,Ownership,Region,Source,Ownership_date"
0,"Иванов И.И.,ООО ""Ромашка"",7701234567,50%,Москв..."
1,"Иванов Иван Иванович,ООО Ромашка,7701234567,0...."
2,"Сидоров С.С,ООО ""Ромашка"",7701234567,50 %,Моск..."
3,"Петров Петр Петрович,ООО ""Лютик"",7801234567,10..."
4,"Петров П.П.,ООО ""Лютик"",7801234567,30%,СПб,For..."


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 1 columns):
 #   Column                                                                     Non-Null Count  Dtype 
---  ------                                                                     --------------  ----- 
 0   FIO_owner,Company_name,INN_company,Ownership,Region,Source,Ownership_date  13 non-null     object
dtypes: object(1)
memory usage: 236.0+ bytes


In [10]:
df.columns

Index(['FIO_owner,Company_name,INN_company,Ownership,Region,Source,Ownership_date'], dtype='object')

## Для начала необходимо разбить стролбец на нормальные поля.

In [12]:
df = df.iloc[:, 0].str.split(',', expand=True) # разбиваем по запятой

# корректные названия для столбцов
df.columns = [
    'FIO_owner',
    'Company_name',
    'INN_company',
    'Ownership',
    'Region',
    'Source',
    'Ownership_date'
]

In [14]:
df.head()

,FIO_owner,Company_name,INN_company,Ownership,Region,Source,Ownership_date
0,Иванов И.И.,"ООО ""Ромашка""",7701234567,50%,Москва,Kommersant,12.03.2021
1,Иванов Иван Иванович,ООО Ромашка,7701234567,0.5,Москва,Kommersant,2021-03-12
2,Сидоров С.С,"ООО ""Ромашка""",7701234567,50 %,Москва,,15/03/2021
3,Петров Петр Петрович,"ООО ""Лютик""",7801234567,100%,Санкт-Петербург,RBC,01-04-2020
4,Петров П.П.,"ООО ""Лютик""",7801234567,30%,СПб,Forbes,2022/05/10


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   FIO_owner       13 non-null     object
 1   Company_name    13 non-null     object
 2   INN_company     13 non-null     object
 3   Ownership       13 non-null     object
 4   Region          13 non-null     object
 5   Source          13 non-null     object
 6   Ownership_date  13 non-null     object
dtypes: object(7)
memory usage: 860.0+ bytes


## 1. Этап очистки и нормализации

### Привести ФИО к формату: Фамилия И.О.

In [18]:
df['FIO_owner']

0                    Иванов И.И.
1           Иванов Иван Иванович
2                    Сидоров С.С
3           Петров Петр Петрович
4                    Петров П.П.
5                    Иванов И.И.
6                   Смирнов А.А.
7     Смирнов Алексей Алексеевич
8                 Кузнецова Е.В.
9                     Орлов Д.Д.
10                   Орлов Д. Д.
11                  Лебедев М.М.
12                   Иванов И.И.
Name: FIO_owner, dtype: object

In [20]:
def normalize_fio(fio):
    if pd.isna(fio):
        return None

    fio = str(fio).strip() # убрать лишние пробеолы в начале и в конце

    fio = " ".join(fio.split()) # убрать лишние пробелы внутри строки
    
    parts = fio.split()
    surname = parts[0].capitalize()

    # обработка случаев с инициалами
    if len(parts) == 2:
        initials = parts[1].replace(" ", "").split(".")
        initials = [i for i in initials if i] # убрать пустые элементы
        
        if len(initials) >=2:
            return f"{surname} {initials[0][0].upper()}.{initials[1][0].upper()}."
        elif len(initials) == 1:
            return f"{surname} {initials[0][0].upper()}."

    # обработка случаев Фамилия Имя Отчество
    if len(parts) >= 3:
        name_initial = parts[1][0].upper()
        patronymic_initial = parts[2][0].upper()
        return f"{surname} {name_initial}.{patronymic_initial}."

    return surname

In [22]:
df['FIO_owner'] = df['FIO_owner'].apply(normalize_fio)

In [24]:
df['FIO_owner']

0        Иванов И.И.
1        Иванов И.И.
2       Сидоров С.С.
3        Петров П.П.
4        Петров П.П.
5        Иванов И.И.
6       Смирнов А.А.
7       Смирнов А.А.
8     Кузнецова Е.В.
9         Орлов Д.Д.
10        Орлов Д.Д.
11      Лебедев М.М.
12       Иванов И.И.
Name: FIO_owner, dtype: object

### Привести названия компаний к единому виду (без кавычек, без лишних пробелов)

In [26]:
df['Company_name']

0     ООО "Ромашка"
1       ООО Ромашка
2     ООО "Ромашка"
3       ООО "Лютик"
4       ООО "Лютик"
5       ООО "Лютик"
6       АО "Вектор"
7         АО Вектор
8       ООО "Альфа"
9       ООО "Альфа"
10        ООО Альфа
11      ООО "Гамма"
12      ООО "Гамма"
Name: Company_name, dtype: object

In [28]:
df['Company_name'] = (
    df['Company_name']
    .str.replace('"', '')
    .str.replace("'", "")
    .str.strip()
)

In [30]:
df['Company_name']

0     ООО Ромашка
1     ООО Ромашка
2     ООО Ромашка
3       ООО Лютик
4       ООО Лютик
5       ООО Лютик
6       АО Вектор
7       АО Вектор
8       ООО Альфа
9       ООО Альфа
10      ООО Альфа
11      ООО Гамма
12      ООО Гамма
Name: Company_name, dtype: object

### Привести ИНН к строковому типу + найти записи с отсутствующим ИНН и обозначить их отдельно


In [32]:
df["INN_company"]

0     7701234567
1     7701234567
2     7701234567
3     7801234567
4     7801234567
5     7801234567
6     5401234567
7     5401234567
8               
9     7723456789
10    7723456789
11    6601234567
12    6601234567
Name: INN_company, dtype: object

In [34]:
df['INN_company'] = (
    df['INN_company']
    .astype('string')
    .str.strip()
    .replace('',pd.NA)
)

In [36]:
missing_inn = df[df['INN_company'].isna()]

print("Записи без ИНН:")
print(missing_inn)

Записи без ИНН:
        FIO_owner Company_name INN_company Ownership  Region Source  \
8  Кузнецова Е.В.    ООО Альфа        <NA>       40%  Москва    RBC   

  Ownership_date  
8     05.06.2020  


In [38]:
df['INN_company']

0     7701234567
1     7701234567
2     7701234567
3     7801234567
4     7801234567
5     7801234567
6     5401234567
7     5401234567
8           <NA>
9     7723456789
10    7723456789
11    6601234567
12    6601234567
Name: INN_company, dtype: string

### Привести Ownership к единому формату (в процентах (float), убрать пробелы, пустые значения выделить)

In [40]:
df['Ownership']

0      50%
1      0.5
2     50 %
3     100%
4      30%
5     30 %
6      25%
7     0.25
8      40%
9      70%
10     35%
11        
12     60%
Name: Ownership, dtype: object

In [42]:
def normalize_ownership(x):
    if pd.isna(x) or x == "":
        return None

    x = str(x).strip().replace("%", "")

    try:
        value = float(x)

        # если доля меньше 1, то считаем что это 0.5 = 50%
        if value <= 1:
            value *= 100

        return round(value, 2)

    except ValueError:
        return None

df['Ownership'] = df['Ownership'].apply(normalize_ownership).astype(float)

In [44]:
missing_ownership= df[df['Ownership'].isna()]

print("Записи без ИНН:")
print(missing_ownership)

Записи без ИНН:
       FIO_owner Company_name INN_company  Ownership        Region  \
11  Лебедев М.М.    ООО Гамма  6601234567        NaN  Екатеринбург   

        Source Ownership_date  
11  Kommersant     12.11.2021  


In [46]:
df['Ownership']

0      50.0
1      50.0
2      50.0
3     100.0
4      30.0
5      30.0
6      25.0
7      25.0
8      40.0
9      70.0
10     35.0
11      NaN
12     60.0
Name: Ownership, dtype: float64

### Привести даты к единому формату: YYYY-MM-DD

In [48]:
df['Ownership_date']

0     12.03.2021
1     2021-03-12
2     15/03/2021
3     01-04-2020
4     2022/05/10
5     10.05.2022
6     2019-07-01
7     01.07.2019
8     05.06.2020
9     2020-06-05
10    2021-08-01
11    12.11.2021
12    2021.11.12
Name: Ownership_date, dtype: object

In [50]:
print(df['Ownership_date'].apply(type))

0     <class 'str'>
1     <class 'str'>
2     <class 'str'>
3     <class 'str'>
4     <class 'str'>
5     <class 'str'>
6     <class 'str'>
7     <class 'str'>
8     <class 'str'>
9     <class 'str'>
10    <class 'str'>
11    <class 'str'>
12    <class 'str'>
Name: Ownership_date, dtype: object


In [54]:
df['Ownership_date'] = pd.to_datetime(
    df['Ownership_date'], 
    dayfirst=True,
    format='mixed',
    errors='coerce'
)

In [56]:
df['Ownership_date']

0    2021-03-12
1    2021-03-12
2    2021-03-15
3    2020-04-01
4    2022-05-10
5    2022-05-10
6    2019-07-01
7    2019-07-01
8    2020-06-05
9    2020-06-05
10   2021-08-01
11   2021-11-12
12   2021-11-12
Name: Ownership_date, dtype: datetime64[ns]

### Итоговый преобразованный датафрейм

In [58]:
df

,FIO_owner,Company_name,INN_company,Ownership,Region,Source,Ownership_date
0,Иванов И.И.,ООО Ромашка,7701234567,50.0,Москва,Kommersant,2021-03-12
1,Иванов И.И.,ООО Ромашка,7701234567,50.0,Москва,Kommersant,2021-03-12
2,Сидоров С.С.,ООО Ромашка,7701234567,50.0,Москва,,2021-03-15
3,Петров П.П.,ООО Лютик,7801234567,100.0,Санкт-Петербург,RBC,2020-04-01
4,Петров П.П.,ООО Лютик,7801234567,30.0,СПб,Forbes,2022-05-10
5,Иванов И.И.,ООО Лютик,7801234567,30.0,Москва,Forbes,2022-05-10
6,Смирнов А.А.,АО Вектор,5401234567,25.0,Новосибирск,Kommersant,2019-07-01
7,Смирнов А.А.,АО Вектор,5401234567,25.0,Новосибирск,,2019-07-01
8,Кузнецова Е.В.,ООО Альфа,<NA>,40.0,Москва,RBC,2020-06-05
9,Орлов Д.Д.,ООО Альфа,7723456789,70.0,Москва,Kommersant,2020-06-05


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   FIO_owner       13 non-null     object        
 1   Company_name    13 non-null     object        
 2   INN_company     12 non-null     string        
 3   Ownership       12 non-null     float64       
 4   Region          13 non-null     object        
 5   Source          13 non-null     object        
 6   Ownership_date  13 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(4), string(1)
memory usage: 860.0+ bytes


## 2. Анализ данных

### Найти компании, где суммарная доля владельцев > 100%

In [70]:
# Суммируем доли по компании
company_share_sum = (
    df.groupby('Company_name', dropna=False)['Ownership']
    .sum(min_count=1)
    .reset_index()
)

# Фильтруем те, что > 100%
over_100 = company_share_sum[company_share_sum['Ownership'] > 100]

over_100

,Company_name,Ownership
1,ООО Альфа,145.0
3,ООО Лютик,160.0
4,ООО Ромашка,150.0


### Найти компании, где доли менялись во времени

Если у компании у одного и того же владельца разные доли на разные даты - значит доля менялась.

In [74]:
# Количество уникальных долей по владельцу внутри компании
ownership_changes = (
    df.groupby(['Company_name', 'FIO_owner'])['Ownership']
    .nunique()
    .reset_index()
)

# Оставляем случаи, где доля менялась
changed = ownership_changes[ownership_changes['Ownership'] > 1]

changed

,Company_name,FIO_owner,Ownership
2,ООО Альфа,Орлов Д.Д.,2
6,ООО Лютик,Петров П.П.,2


### Найти владельцев, участвующих более чем в одной компании

In [78]:
owner_multiple_companies = (
    df.groupby('FIO_owner')['Company_name']
    .nunique()
    .reset_index()
)

owner_multiple_companies = owner_multiple_companies[
    owner_multiple_companies['Company_name'] > 1
]

owner_multiple_companies

,FIO_owner,Company_name
0,Иванов И.И.,3


In [80]:
df[df['FIO_owner'].isin(owner_multiple_companies['FIO_owner'])]

,FIO_owner,Company_name,INN_company,Ownership,Region,Source,Ownership_date
0,Иванов И.И.,ООО Ромашка,7701234567,50.0,Москва,Kommersant,2021-03-12
1,Иванов И.И.,ООО Ромашка,7701234567,50.0,Москва,Kommersant,2021-03-12
5,Иванов И.И.,ООО Лютик,7801234567,30.0,Москва,Forbes,2022-05-10
12,Иванов И.И.,ООО Гамма,6601234567,60.0,Екатеринбург,,2021-11-12
